In [3]:
#!pip install -U huggingface_hub scikit-learn pandas numpy --quiet


In [4]:
import os
from huggingface_hub import InferenceClient


from google.colab import userdata


os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')



client = InferenceClient(
    model="deepseek-ai/DeepSeek-R1",
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
)



In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

def generate_metrics():
    df = pd.DataFrame({
        "cpu": np.random.normal(50, 10, 100),
        "memory": np.random.normal(60, 15, 100),
        "disk_io": np.random.normal(100, 25, 100),
    })
    df.iloc[95:100] += np.random.uniform(100, 200)
    return df

def detect_anomalies(df):
    model = IsolationForest(contamination=0.05, random_state=42)
    model.fit(df)
    df["anomaly"] = model.predict(df)
    return df[df["anomaly"] == -1]


In [6]:
def build_messages(anomalies_df):
    return [
        {"role": "system", "content": "You are a senior DevOps engineer helping to explain system anomalies."},
        {"role": "user", "content": f"""Anomalies were detected in the following system metrics:

{anomalies_df.to_string(index=False)}

Explain:
1. What might have caused these?
2. What is the likely impact?
3. How should we fix it?
"""}
    ]


In [7]:
import time

def evaluate_response(text, start_time):
    duration = round(time.time() - start_time, 2)
    words = len(text.split())
    keywords = ["cpu", "memory", "disk", "restart", "fix", "usage", "overload"]
    relevance_hits = sum(k in text.lower() for k in keywords)
    relevance_score = round(relevance_hits / len(keywords), 2)
    return {
        "latency_seconds": duration,
        "word_count": words,
        "relevance_score": relevance_score
    }


In [8]:
df = generate_metrics()
anomalies = detect_anomalies(df)

if anomalies.empty:
    print("✅ No anomalies detected.")
else:
    print("🚨 Anomalies found. Sending to DeepSeek...\n")
    messages = build_messages(anomalies)

    start = time.time()
    response = client.chat_completion(messages=messages)
    reply = response.choices[0].message.content.strip()

    print(" DeepSeek Response:\n", reply)

    metrics = evaluate_response(reply, start)
    print("\n Evaluation Metrics:")
    for k, v in metrics.items():
        print(f" - {k}: {v}")


🚨 Anomalies found. Sending to DeepSeek...



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


 DeepSeek Response:
 <think>
Okay, let me try to work through this problem. So, there's a dataset with system metrics for CPU, memory, disk I/O, and an anomaly flag. All the rows here are marked as anomalies with -1. My task is to figure out what's causing these anomalies, their impact, and how to fix them. Let's break it down step by step.

First, looking at the numbers. The CPU seems to be in the 237-253 range, memory is from about 242 to 279, disk I/O is the highest at around 273-318. Wait, the units aren't specified, which could be an issue. But typically, CPU usage is a percentage, so maybe here it's scaled? If those are percentages, CPU at 237% would indicate a system with more than one core since 100% per core is max. So if it's a multi-core system, maybe each CPU is maxed out. But 237% might mean 2.37 cores fully utilized. But that's still high. Memory here, if it's in MB or GB, but 242 seems high if it's GB. Maybe it's in MB, which would be 242MB, which is low. Wait, but then 